# Day 10 - 머신러닝(4)

In [3]:
!pip install category_encoders

  Obtaining dependency information for category_encoders from https://files.pythonhosted.org/packages/1f/e2/495811f12b2e90753fff0e42a07adb0370a725de17cc23a579ac9d3ca67c/category_encoders-2.6.2-py2.py3-none-any.whl.metadata
  Obtaining dependency information for scikit-learn>=0.20.0 from https://files.pythonhosted.org/packages/f1/7d/2e562207176a5dcdad513085670674bb11ffaf37e1393eacb6d7fb502481/scikit_learn-1.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scipy>=1.0.0 from https://files.pythonhosted.org/packages/81/d7/d2537d51efb692d0c411e64267ba349e7668d40f5bc73cefe78ccd650dcd/scipy-1.11.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.4 kB 326.8 kB/s eta 0:00:01
     -------------------------------------- 60.4/60.4 kB 355.8 kB/s eta 0:00:00
     -----------------------------

In [4]:
import numpy as np 
import pandas as pd

import category_encoders as ce # encoding 모듈 사용

- 범주형 데이터를 연속형,이산형 데이터와 다르게 표현할 수 있는지에 대한 테크닉
- 범주형 데이터 => 숫자형 데이터 
    - 순서형 데이터를 인코딩안하고 명목형 데이터만 인코딩하면 이산형 데이터로 인식됨
    - 순서형과 명목형 데이터 인코딩법이 다르기도 함.(선택적)
- cleaning 후 encoding
<br>
<br>
- 데이터 인코딩 분류
    - Norminal Encoding : 순서형
        - One hot encoding : 크기표현 x (딥러닝에서 많이 사용하는 기법) / nunique개수가 적으면 사용<br>
            - 결측치가 무조건 없어야 실행가능<br>
            - red가 green보다 3배 작다라는 크기비교를 안하게 하는것(장점:명확하게 구분가능)<br>
            - 데이터가 3개인데 위 기법을 사용하면 연산을 9번해야함.(기하급수적으로 늘어남)<br>
                => 메모리 차지가 커진다.(단점)<br>
        - Mean Encoding : 각각의 요소에 대한 평균값으로 인코딩<br>
<br>
    - Ordinal Encoding : 명목형<br>
        - label Encoding : computer가 label을 인식하든 말든 endoing실행<br>
        - Target Encoding : target을 모듈(target을 기준으로 평균 / mean+labe encoding)<br>
        - Ordinal Encoding : 순서매기기 (label이랑 비슷함)



In [5]:
data = {'color': ['Red', 'Blue', 'Green']}
df = pd.DataFrame(data) 
df.head()

,color
0,Red
1,Blue
2,Green


In [6]:
# encoder 인스턴스 생성
# 인스턴스에는 변수와 함수가 있다
encoder = ce.OneHotEncoder(use_cat_names=True) # 인코딩 객체 만들어짐.

# 학습
encoder.fit(df)
# 반영
# transform을 사용하기전에 꼭 fit()을 실행해야한다. (fit 먼저 실행 후 transform실행)
df_encoded = encoder.transform(df)
# => df_encoded = encoder.fit_transform(df) # 인코딩 객체가 fit, transform....

df_encoded.head() # transform 결과 보기

c:\dev\pandas\.venv\Lib\site-packages\category_encoders\utils.py:28: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(cols):
c:\dev\pandas\.venv\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
c:\dev\pandas\.venv\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.types.is_categorical_dtype(dtype)
c:\dev\pandas\.venv\Lib\site-packages\category_encoders\utils.py:50: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return pd.api.type

,color_Red,color_Blue,color_Green
0,1,0,0
1,0,1,0
2,0,0,1


***
#### 예제

In [7]:
import seaborn as sns

df = sns.load_dataset('titanic')
cols = ["age","sibsp","parch","fare","pclass","sex","embarked", "survived"]

df = df[cols]
df.shape 

(891, 8)

In [8]:
from sklearn.model_selection import train_test_split
SEED=42

train, test = train_test_split(df, random_state=SEED, test_size=0.2)

train.shape, test.shape

((712, 8), (179, 8))

In [9]:
# 결측치 제거
train.age = train.age.fillna(train.age.mean())
test.age = test.age.fillna(train.age.mean())

train['embarked'] = train.embarked.fillna(train.embarked.mode().values[0])
test.embarked = test.embarked.fillna(train.embarked.mode().values[0])

train.isnull().sum().sum()

0

In [10]:
cols = ["age","fare"]
features_tr = train[cols]
target_tr = train["survived"]
features_te = test[cols]
target_te = test["survived"]

features_tr.shape, target_tr.shape
# 수치형 분리

((712, 2), (712,))

In [11]:
cols_encoding = ["pclass","sex","embarked","sibsp","parch"]
tmp_tr = train[cols_encoding]
tmp_te = test[cols_encoding]

tmp_tr.shape 
# 범주형만 분리해서 encoding / train, test 파일 둘다 encoding

(712, 5)

In [ ]:
enco_tr = pd.DataFrame()
enco_te = pd.DataFrame()

for col in tmp_tr.columns:
#학습
    encoder.fit_transform(tmp_tr[col].astype('category'))

# 반영
# train 반영
    _enco = encoder.transform(tmp_tr[col].astype('category'))

# 기존 데이터에 인코딩된 데이터를 추가
    enco_tr = pd.concat([enco_tr, _enco], axis=1)

# test 반영
    _enco = encoder.transform(tmp_te[col].astype('category'))
    
    enco_te = pd.concat([enco_te, _enco], axis=1) #기존 데이터에 인코딩된 데이터를 추가

print(f'{enco_tr.shape} / {enco_te.shape}')
enco_tr.head()